In [2]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

In [3]:
import gempy as gp
import gempy_viewer as gpv

Setting Backend To: AvailableBackends.numpy


In [4]:
data_path = 'https://raw.githubusercontent.com/cgre-aachen/gempy_data/master/'
path_to_data = data_path + "/data/input_data/jan_models/"


geo_model: gp.data.GeoModel = gp.create_geomodel(
    project_name='fold',
    extent=[6512290.17, 6512361.87, 5690041.29, 5690094.19, 114.85, 145.35],
    refinement=6,
    importer_helper=gp.data.ImporterHelper(
        path_to_orientations="D:\Programy experymentalne\BDI_model_3D\model2_orientations.csv",
        path_to_surface_points="D:\Programy experymentalne\BDI_model_3D\model2_surface_points.csv"
    )
)


gp.map_stack_to_surfaces(
    gempy_model=geo_model,
    mapping_object={
        "Strat_Series": ("Cl1", "Cl2", "Cl4", "Gr3", "Hu", "orSa1b", "orSi2", "Pt", "Sa1a", "Sa1b", "Sa2", "Sa3", "Sa4b", "Sa5", "Sa6", "Si1", "Si2", "Si3", "Si4",'\\N')  
    }
)

gp.compute_model(geo_model)
geo_data = geo_model

Setting Backend To: AvailableBackends.numpy


c:\Users\Uzytkownik\AppData\Local\Programs\Python\Python311\Lib\site-packages\gempy_engine\modules\activator\_soft_segment.py:95: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + bt.t.exp(x))


Chunking done: 34 chunks
Chunking done: 31 chunks
Chunking done: 240 chunks
Chunking done: 213 chunks
Chunking done: 45 chunks


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

# --- rysowanie przekroju ---
p = gpv.plot_2d(
    geo_data,
    direction=['y'],
    show_data=True,
    legend=False,
    ve=1
)

ax = p.axes[0]
fig = ax.figure

# --- zakres danych ---
xmin, ymin, w, h = ax.dataLim.bounds
xmax = xmin + w
zmin = ymin
zmax = ymin + h

profile_length_m = xmax - xmin

# --- SKALE INŻYNIERSKIE ---
scale_h = 500   # 1:500 poziom
scale_v = 100   # 1:100 pion

left_mm, right_mm = 20, 20
bottom_mm, top_mm = 0, 0

print(profile_length_m)

sheet_width_mm = (profile_length_m * (1000 / scale_h))
sheet_height_mm = 297

left   = left_mm / (sheet_width_mm + left_mm + right_mm)
right  = right_mm / (sheet_width_mm + left_mm + right_mm)
bottom = bottom_mm / sheet_height_mm
top    = top_mm / sheet_height_mm

print(left)
print(right)
print(bottom)
print(top)


# --- ROZMIAR ARKUSZA (mm) ---
# sheet_width_mm = (profile_length_m * (1000 / scale_h)) + left_mm + right_mm
                                 # np. A3 / A1-rolka

fig.set_size_inches(
    (sheet_width_mm + left_mm + right_mm) / 25.4,
    (sheet_height_mm + bottom_mm + top_mm) / 25.4
)




width  = 1 - left - right
height = 1 - bottom - top

ax.set_position([left, bottom, width, height])

# --- POZIOM: CAŁY PROFIL (skala z szerokości kartki) ---
ax.set_xlim(xmin, xmax)

# --- PION: ZAKRES Z FIZYCZNEJ WYSOKOŚCI OSI ---
fig_height_in = fig.get_size_inches()[1]
axis_height_mm = fig_height_in * height * 25.4

y_range_m = axis_height_mm / (1000 / scale_v)  # np. 10 mm = 1 m
z_center = (zmin + zmax) / 2

ax.set_ylim(z_center + y_range_m / 2,
            z_center - y_range_m / 2)

ax.invert_yaxis()

# --- TICKI ---
ax.xaxis.set_major_locator(MultipleLocator(50))
ax.xaxis.set_minor_locator(MultipleLocator(10))
ax.yaxis.set_major_locator(MultipleLocator(10))
ax.yaxis.set_minor_locator(MultipleLocator(1))

ax.tick_params(axis='both', which='major', length=8, width=1.2)
ax.tick_params(axis='both', which='minor', length=4, width=0.8)

# --- RAMKA ---
for spine in ax.spines.values():
    spine.set_linewidth(2.0)
    spine.set_visible(True)

# --- usuń colorbar ---
for a in fig.axes:
    if a is not ax:
        fig.delaxes(a)

# --- zapis ---
fig.savefig("przekroj_1_500_1_100.pdf", dpi=300)
plt.close(fig)


71.70000000018626
0.06119951040377724
0.06119951040377724
0.0
0.0


c:\Users\Uzytkownik\AppData\Local\Programs\Python\Python311\Lib\site-packages\gempy_viewer\API\_plot_2d_API.py:176: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  p.fig.show()


In [8]:
gpv.plot_3d(geo_data, show_surfaces=True)

ValueError: path is on mount 'c:', start on mount 'd:'